In [1]:
import os

In [23]:
import sys
sys.path.append('/Users/admin/HB_IDS/HackBeta/src')

In [2]:
os.chdir('../')

In [3]:
pwd

'/Users/admin/HB_IDS/HackBeta'

In [16]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_file_path: Path
    local_data_file: Path
           
  

In [17]:
from hackbeta.constants import *
from hackbeta.utils.common import read_yaml,create_directories

In [18]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the root directory for artifacts as defined in the YAML configuration
        create_directories([self.config['artifacts_root']])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']

        # Ensure the data ingestion root directory exists
        create_directories([config['root_dir']])

        # Create a DataIngestionConfig object with paths from the YAML configuration
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'],
            source_file_path=config['source_file_path'],  # Path to the source CSV file
            local_data_file=config['local_data_file'],    # Target path for the moved CSV file
        )

        return data_ingestion_config


In [19]:
import os
import shutil
import urllib.request as request
import zipfile
from hackbeta.logging import logger
from hackbeta.utils.common import get_size


class DataIngestion:
    def __init__(self, config):
        self.config = config

    def move_csv_file(self):
        """
        Moves the CSV file from a source location to the target location
        specified in the configuration. If the file already exists at the target
        location, the function logs a message and does not overwrite the file.
        """
        source_file_path = Path(self.config.source_file_path)  # Path to the source CSV file
        target_file_path = Path(self.config.local_data_file)  # Target location as per configuration

        # Ensure the target directory exists
        target_file_path.parent.mkdir(parents=True, exist_ok=True)

        if not source_file_path.exists():
            logger.error(f"Source CSV file does not exist at {source_file_path}")
            return

        if target_file_path.exists():
            logger.info(f"File already exists at target location: {target_file_path}")
            return

        try:
            shutil.move(str(source_file_path), str(target_file_path))
            logger.info(f"CSV file moved successfully from {source_file_path} to {target_file_path}")
        except Exception as e:
            logger.error(f"Failed to move CSV file: {e}")


In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.move_csv_file()  
except Exception as e:
    raise e


[2024-04-06 20:54:56,340: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-06 20:54:56,372: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-06 20:54:56,378: INFO: common: created directory at: artifacts]
[2024-04-06 20:54:56,380: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-06 20:54:56,392: INFO: 3907264586: CSV file moved successfully from /Users/admin/Downloads/Leather_Mag_Text_Cleaned.csv to artifacts/data_ingestion/Leather_Mag_Text_Cleaned.csv]
